In [0]:
%pip install mlflow==3.1.0
%pip install azure-core azure-storage-file-datalake #for the display() in Azure only
dbutils.library.restartPython()

#Registering python UDF to a SQL function
This is a companion notebook to load the `predict_churn` model as a spark udf and save it as a SQL function. While this code was present in the SQL notebook, it won't be run by the DLT engine (since the notebook is SQL we only read sql cess)
 
For the UDF to be available, you must this notebook in your DLT. (Currently mixing python in a SQL DLT notebook won't run the python)

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=1444828305810485&notebook=%2F01-Data-ingestion%2F01.2-DLT-churn-Python-UDF&demo_name=lakehouse-retail-c360&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-retail-c360%2F01-Data-ingestion%2F01.2-DLT-churn-Python-UDF&version=1&user_hash=0b3be070fa39374fb760232ebb606a5c489732ec881a7ebfc68231c496aed118">

In [0]:
import mlflow 
mlflow.set_registry_uri('databricks-uc')
#                                                                                                     Stage/version  
#                                                                                   Model name               |        
#                                                                                       |                    |        
predict_churn_udf = mlflow.pyfunc.spark_udf(spark, "models:/data_pioneers.c360.dbdemos_customer_churn@prod", "long", env_manager='virtualenv')
spark.udf.register("predict_churn", predict_churn_udf)

### Setting up the DLT 

This notebook must be included in your DLT "libraries" parameter:

```
{
    "id": "95f28631-1884-425e-af69-05c3f397dd90",
    "name": "xxxx",
    "storage": "/demos/dlt/lakehouse_churn/xxxxx",
    "configuration": {
        "pipelines.useV2DetailsPage": "true"
    },
    "clusters": [
        {
            "label": "default",
            "autoscale": {
                "min_workers": 1,
                "max_workers": 5
            }
        }
    ],
    "libraries": [
        {
            "notebook": {
                "path": "/Repos/xxxx/01.2-DLT-churn-Python-UDF"
            }
        },
        {
            "notebook": {
                "path": "/Repos/xxxx/01.1-DLT-churn-SQL"
            }
        }
    ],
    "target": "retail_lakehouse_churn_xxxx",
    "continuous": false,
    "development": false
}
```